# Import libraries

In [45]:
import pandas as pd
import urllib.request, json, csv
import requests
from json import JSONEncoder

# Load data

In [46]:
url_dataset = "https://raw.githubusercontent.com/onaio/ona-tech/master/data/water_points.json"
with urllib.request.urlopen(url_dataset) as url:
  # load the data to pandas
    df = pd.read_json(url.read())
df.head()

,_attachments,_bamboo_dataset_id,_deleted_at,_geolocation,_id,_status,_submission_time,_uuid,_xform_id_string,animal_number,...,water_mechanism_plate_units,water_not_functioning,water_pay,water_point_condition,water_point_geocode,water_point_id,water_point_image,water_source_type,water_source_type_other,water_used_season
0,[north_ghana/attachments/1351696546452.jpg],,NaT,"[10.1892764, -0.66410362]",381705,submitted_via_web,2012-11-13 07:13:57,f8bcee72d7a0400fb99ae11bbf804010,_08_Water_points_CV,more_500,...,NaN,NaN,no,functioning,10.1892764 -0.66410362 155.10000610351563 5.0,xxx,1351696546452.jpg,dam_dugout,NaN,year_round
1,[north_ghana/attachments/1351701849971.jpg],,NaT,"[10.28173052, -0.56901122]",381706,submitted_via_web,2012-11-13 07:14:04,c2f6b298955f47ab9f177bee1214141d,_08_Water_points_CV,50_to_500,...,NaN,NaN,no,functioning,10.28173052 -0.56901122 201.89999389648438 5.0,xxx,1351701849971.jpg,unprotected_well,NaN,year_round
2,[north_ghana/attachments/1351702462336.jpg],,NaT,"[10.28169238, -0.56962993]",381707,submitted_via_web,2012-11-13 07:14:07,6bc6d188611d47f6a666cfd1eaa33998,_08_Water_points_CV,50_to_500,...,NaN,NaN,no,functioning,10.28169238 -0.56962993 202.60000610351563 5.0,xxx,1351702462336.jpg,borehole,NaN,year_round
3,[north_ghana/attachments/1351702971561.jpg],,NaT,"[10.28115661, -0.56918339]",381708,submitted_via_web,2012-11-13 07:14:14,4b28ac4cbba744d79ba4257f772f94d6,_08_Water_points_CV,50_to_500,...,NaN,NaN,no,functioning,10.28115661 -0.56918339 199.6999969482422 5.0,xxx,1351702971561.jpg,borehole,NaN,year_round
4,[north_ghana/attachments/1351703622326.jpg],,NaT,"[10.28044635, -0.56723556]",381709,submitted_via_web,2012-11-13 07:14:22,7893ce5321804f229e533f36e90c9c6f,_08_Water_points_CV,50_to_500,...,NaN,NaN,no,functioning,10.28044635 -0.56723556 208.6999969482422 5.0,xxx,1351703622326.jpg,borehole,NaN,year_round


# Relevant columns

In [47]:
df= df[["communities_villages", "water_point_condition", "water_functioning"]]
df.tail()
#df.size

,communities_villages,water_point_condition,water_functioning
707,Suik,functioning,yes
708,Vundema,broken,yes
709,Vundema,broken,yes
710,Jiniensa,functioning,yes
711,Jagsa,functioning,yes


In [48]:
df['water_point_condition'].value_counts()

functioning           590
broken                 40
abandoned              36
newly_constructed      28
under_construction     16
na_dn                   2
Name: water_point_condition, dtype: int64

# Create Class

In [49]:
class waterPoints:
  #initialize the class attributes
  def __init__(self,data):
    # load data from the url
    self.data = data 
   #The number of water points that are functional
  def functional_water_points (self):
    #filter the data by functional water points
    number_functional=self.data[self.data['water_point_condition'] =="functioning"]
    self.total_functional = len(number_functional)

    self.number_functional= number_functional.groupby('water_point_condition').size().reset_index(name="number_functional")
    print("1: The number of water points that are functional:")
    print()
    print(self.number_functional)

  #Calculate number of water points per community
  def community_waterpoints (self):
    self.community_water_points= self.data.groupby("communities_villages").size().reset_index(name='total_water_points')
    print("2: Number of water points per community: ")
    print()
    print(self.community_water_points)

  #The rank for each community by the percentage of broken water points
  def broken_water_points (self):
    #filter by broken points
    bwp = self.data[self.data['water_point_condition'] =="broken"]
    #Convert to dataframe
    self.broken_community_water_points= bwp.groupby("communities_villages").size().reset_index(name='broken_water_points')
    # merge dataframes to get data of broken water points
    self.water_points_merged = pd.merge(self.community_water_points,self.broken_community_water_points, how='outer', on='communities_villages')
    self.water_points_merged['broken_water_points'] = self.water_points_merged['broken_water_points'].fillna(0)

    # # calculate the percentage of broken water points and rank
    self.water_points_merged['broken_water_points_percentage'] = round((self.water_points_merged['broken_water_points'] / self.water_points_merged['total_water_points']) *100, 2)
    self.water_points_merged['rank'] = self.water_points_merged['broken_water_points_percentage'].rank(method='first', ascending=False).astype(int)
    self.water_points_merged.sort_values(by='broken_water_points_percentage', inplace=True, ascending=False)

    print("3: Rank for each community by the percentage of broken water points. ")
    print()
    print(self.water_points_merged)

  # store data in dictionary
  def data_analysis(self):
    data_set = {
        'number_functional': self.total_functional,
        'number_water_points': self.community_water_points.to_dict('records'),
        'community_ranking': self.water_points_merged[['communities_villages', 'broken_water_points_percentage', 'rank']].to_dict('records')
        }
    
  # save data to file
    with open("analysis.json", "w", encoding ='utf8') as outfile:
        json.dump(data_set, outfile, indent=4, sort_keys=False)

In [50]:
# instantiate the class
obj=waterPoints(df)

print()
#Calling methods by created object
obj.functional_water_points()
print()
obj.community_waterpoints()
print()
obj.broken_water_points()


1: The number of water points that are functional:

  water_point_condition  number_functional
0           functioning                590

2: Number of water points per community: 

   communities_villages  total_water_points
0              Abanyeri                   4
1           Akpari-yeri                   3
2              Alavanyo                   3
3                 Arigu                  12
4               Badomsa                  27
5                Bandem                   7
6             Banyangsa                  10
7              Bechinsa                  26
8           Chanpolinsa                   4
9                Chansa                   9
10             Chondema                   4
11               Dibisi                   2
12              Dorinsa                  17
13                Fiisa                   5
14               Gaadem                   2
15               Garigu                   1
16             Gbaarigu                   5
17                Gbima  

# Return data in Json format

In [51]:
obj.data_analysis()